In [1]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from collections import Counter

import Summarizer
import summarizer_data_utils
import summarizer_model_utils

print(tf.__version__)

D:\Softwares\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0324 21:08:47.461876 14040 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


1.10.0


In [2]:
# load csv file using pandas.
file_path = './Data.csv'
data = pd.read_csv(file_path)
data.shape

(5, 2)

In [3]:
data.head()

,Text,Summary
0,Republican vote dissatisfied by appointing adm...,There is a dissatisfaction in the Republican v...
1,"Aam Aadmi Party's rebel leader, Shanti Bhushan...","Aam Aadmi Party's rebel leader, Shanti Bhushan..."
2,It will take time to get good days - LK Advani...,It will take time to get good days - LK Advani...
3,Pak violates ceasefire once again In Poonch di...,Pak violates ceasefire once again In Poonch di...
4,"Make Maharashtra four pieces, say ma ga vaida,...","If the Sangh's role is to break Maharashtra, t..."


In [4]:
# we will not use all of them, only short ones and ones of similar size. 
# choosing the ones that are of similar length makes it easier for the model to learn.
raw_texts = []
raw_summaries = []

for text, summary in zip(data.Text, data.Summary):
    if 100< len(text) < 2000:
        raw_texts.append(text)
        raw_summaries.append(summary)

In [5]:
len(raw_texts), len(raw_summaries)

(5, 5)

In [6]:
#for t, s in zip(raw_texts[:5], raw_summaries[:5]):
    #print('Text:\n', t, '\n')
    #print('Summary:\n', s, '\n\n')
    
    

In [7]:
# the function gives us the option to keep_most of the characters inisde the texts and summaries, meaning
# punctuation, question marks, slashes...
# or we can set it to False, meaning we only want to keep letters and numbers like here.
processed_texts, processed_summaries, words_counted = summarizer_data_utils.preprocess_texts_and_summaries(
    raw_texts,
    raw_summaries,
    keep_most=False
)

Processing Time:  0.04089164733886719


In [8]:
for t,s in zip(processed_texts[:1], processed_summaries[:1]):
    print('Text\n:', t, '\n')
    print('Summary:\n', s, '\n\n\n')

Text
: ['republican', 'vote', 'dissatisfied', 'by', 'appointing', 'administrator', 'there', 'is', 'a', 'dissatisfaction', 'in', 'the', 'republican', 'vote', 'of', 'mp', 'ramdas', 'athavale', 'due', 'to', 'the', 'appointment', 'of', 'the', 'state', 'government', 's', 'college', 'in', 'mahad', 'college', 'of', 'people', 's', 'education', 'society', 'of', 'babasaheb', 'ambedkar', 'the', 'republican', 'bahujan', 'vidyarthi', 'parishad', 'has', 'protested', 'against', 'the', 'government', 's', 'decision', 'and', 'the', 'state', 'government', 'will', 'have', 'to', 'face', 'its', 'protest', 'dr', 'people', 'from', 'pune', 's', 'education', 'society', 'founded', 'by', 'dr', 'babasaheb', 'ambedkar', 'are', 'from', 'mahad', 'pune', 'aurangabad', 'marathwada', 'konkan', 'and', 'many', 'places', 'in', 'bangalore', 'the', 'office', 'of', 'ambedkar', 'and', 'siddhartha', 'college', 'has', 'its', 'office', 'in', 'mumbai', 'the', 'issue', 'has', 'been', 'going', 'on', 'for', 'a', 'number', 'of', 'year

In [9]:
# create lookup dicts.
# most oft the words only appear only once. 
specials = ["<EOS>", "<SOS>","<PAD>","<UNK>"]
word2ind, ind2word,  missing_words = summarizer_data_utils.create_word_inds_dicts(words_counted,
                                                                       specials = specials)
print(len(word2ind), len(ind2word), len(missing_words))

354 354 0


In [10]:
# the embeddings from tf.hub. 
# embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
embed = hub.Module("https://tfhub.dev/google/Wiki-words-250/1")
emb = embed([key for key in word2ind.keys()])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    embedding = sess.run(emb)

#glove_embeddings_path = './glove/glove.42B.300d.txt'
#embedding_matrix_save_path = './embeddings/my_embedding.npy'
#emb = summarizer_data_utils.create_and_save_embedding_matrix(word2ind,
#                                                       glove_embeddings_path,
#                                                       embedding_matrix_save_path)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0324 21:08:51.105129 14040 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [11]:
embedding.shape

(354, 250)

In [12]:
np.save('./tf_hub_embedding_headlines.npy', embedding)

In [13]:
# converts words in texts and summaries to indices
# it looks like we have to set eos here to False
converted_texts, unknown_words_in_texts = summarizer_data_utils.convert_to_inds(processed_texts,
                                                                                word2ind,
                                                                                eos = False)

converted_summaries, unknown_words_in_summaries = summarizer_data_utils.convert_to_inds(processed_summaries,
                                                                                        word2ind,
                                                                                        eos = True,
                                                                                        sos = True)

#converted_texts[0]

In [14]:
# seems to have worked well. 
print( summarizer_data_utils.convert_inds_to_text(converted_texts[0], ind2word),
       summarizer_data_utils.convert_inds_to_text(converted_summaries[0], ind2word))

['republican', 'vote', 'dissatisfied', 'by', 'appointing', 'administrator', 'there', 'is', 'a', 'dissatisfaction', 'in', 'the', 'republican', 'vote', 'of', 'mp', 'ramdas', 'athavale', 'due', 'to', 'the', 'appointment', 'of', 'the', 'state', 'government', 's', 'college', 'in', 'mahad', 'college', 'of', 'people', 's', 'education', 'society', 'of', 'babasaheb', 'ambedkar', 'the', 'republican', 'bahujan', 'vidyarthi', 'parishad', 'has', 'protested', 'against', 'the', 'government', 's', 'decision', 'and', 'the', 'state', 'government', 'will', 'have', 'to', 'face', 'its', 'protest', 'dr', 'people', 'from', 'pune', 's', 'education', 'society', 'founded', 'by', 'dr', 'babasaheb', 'ambedkar', 'are', 'from', 'mahad', 'pune', 'aurangabad', 'marathwada', 'konkan', 'and', 'many', 'places', 'in', 'bangalore', 'the', 'office', 'of', 'ambedkar', 'and', 'siddhartha', 'college', 'has', 'its', 'office', 'in', 'mumbai', 'the', 'issue', 'has', 'been', 'going', 'on', 'for', 'a', 'number', 'of', 'years', 'as

In [15]:
# model hyperparameters
num_layers_encoder = 4
num_layers_decoder = 4
rnn_size_encoder = 250
rnn_size_decoder = 250

batch_size = 10
epochs = 2
clip = 2
keep_probability = 0.8
learning_rate = 0.0005
max_lr=0.005
learning_rate_decay_steps = 100
learning_rate_decay = 0.90


pretrained_embeddings_path = './tf_hub_embedding_headlines.npy'
summary_dir = os.path.join('./tensorboard/headlines')

use_cyclic_lr = True
inference_targets=True

In [16]:
len(converted_summaries)

5

In [17]:
# build graph and train the model 
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   save_path='./models/headlines/my_model',
                                   mode='TRAIN',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   batch_size = batch_size,
                                   clip = clip,
                                   keep_probability = keep_probability,
                                   learning_rate = learning_rate,
                                   max_lr=max_lr,
                                   learning_rate_decay_steps = learning_rate_decay_steps,
                                   learning_rate_decay = learning_rate_decay,
                                   epochs = epochs,
                                   pretrained_embeddings_path = pretrained_embeddings_path,
                                   use_cyclic_lr = use_cyclic_lr,)
#                                    summary_dir = summary_dir)           

summarizer.build_graph()
summarizer.train(converted_texts, 
                 converted_summaries)

Loaded pretrained embeddings.
Instructions for updating:
seq_dim is deprecated, use seq_axis instead


W0324 21:09:03.450108 14040 tf_logging.py:125] From D:\Softwares\Anaconda\lib\site-packages\tensorflow\python\ops\rnn.py:430: calling reverse_sequence (from tensorflow.python.ops.array_ops) with seq_dim is deprecated and will be removed in a future version.
Instructions for updating:
seq_dim is deprecated, use seq_axis instead


Instructions for updating:
batch_dim is deprecated, use batch_axis instead


W0324 21:09:03.486014 14040 tf_logging.py:125] From D:\Softwares\Anaconda\lib\site-packages\tensorflow\python\util\deprecation.py:454: calling reverse_sequence (from tensorflow.python.ops.array_ops) with batch_dim is deprecated and will be removed in a future version.
Instructions for updating:
batch_dim is deprecated, use batch_axis instead
D:\Softwares\Anaconda\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Graph built.
-------------------- Epoch 0 of 2 --------------------
Iteration: 0 of 0	train_loss: 5.8692
Average Score for this Epoch: 5.869154930114746
--- new best score ---


-------------------- Epoch 1 of 2 --------------------
Iteration: 0 of 0	train_loss: 5.8610
Average Score for this Epoch: 5.861011028289795
--- new best score ---


-------------------- Epoch 2 of 2 --------------------
Iteration: 0 of 0	train_loss: 5.8414
Average Score for this Epoch: 5.841433048248291
--- new best score ---




In [ ]:
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   './models/headlines/my_model',
                                   'INFER',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   batch_size = len(converted_texts[:5]),
                                   clip = clip,
                                   keep_probability = 1.0,
                                   learning_rate = 0.0,
                                   beam_width = 5,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   inference_targets = False,
                                   pretrained_embeddings_path = pretrained_embeddings_path)

summarizer.build_graph()
preds = summarizer.infer(inputs = converted_texts[:5],
                         restore_path =  './models/headlines/my_model',
                         targets = converted_summaries[:5])

Loaded pretrained embeddings.
Graph built.
INFO:tensorflow:Restoring parameters from ./models/headlines/my_model


I0324 21:05:33.629667  1644 tf_logging.py:115] Restoring parameters from ./models/headlines/my_model


Path Restored.
if executed


In [24]:
# show results
summarizer_model_utils.sample_results(preds,
                                      ind2word,
                                      word2ind,
                                      converted_summaries[:5],
                                      converted_texts[:5])

NameError: name 'preds' is not defined